# Autograd: Automatic Differentiation

PyTorch에서 neural network의 중심은 "**autograd**" package이다. 

**autograd** package는 tensor의 모든 작업을 자동으로 구분한다. 이건은 **실행**에 의해서 **정의**되는 프레임워크이다(**define-by-run**). 즉, 내가 실행하는 backpropagation은 내가 작성한 코드가 어떻게 실행되는가에 따라서 정의된다. 그리고 모든 반복이 다 다를 수 있다. 

## Tensor

**torch.Tensor**는 **autograd** 패키지의 중심 class이다. 속성 **.requires_grad**를 **True**로 설정하면 모든 작업을 추적하기 시작한다. 계산이 끝나면 **.backward()**를 호출하고 모든 경사(gradient)를 자동으로 계산할 수 있다. 이 tensor의 경사는 **.grad** 속에 누적된다.

Tensor가 history를 추적하는 것을 막으려면, **.detach()**를 호출하여 계산 history에서 떼어내고, 앞으로의 계산이 추적되는 것을 막을 수 있다. 

History를 추적하는 것을 방지하기 위해서, **with torch.no_grad()**로 코드 블록을 래핑할 수도 있다. 

이 방법은 특히 모델을 평가할 때 유용할 수 있다. 왜냐하면 모델이  **requires_grad = True**로 훈련이 가능한 parameters를 가지고 있을 수 있기 때문이다. 하지만 그것을 위해서 gradient가 필요하지는 않다. 이 방법은 특히 모델을 평가할 때 유용할 수 있다. 왜냐하면 모델이  **requires_grad = True**로 훈련이 가능한 parameters를 가지고 있을 수 있기 때문이다. 하지만 그것을 위해서 gradient가 필요하지는 않다.  

-------------

**autograd**를 구현하기 위해 매우 중요한 클래스가 하나 더 있다. - **Function**

Tensor와 Function은 서로 연결되어 있으며, 완전한 계산 내역을 나타내는 비순환그래프를 만든다. 

각각의 tensor는 그 tensor를 만드는 Function을 참조하는 **.grad_fn**이라는 속성을 가지고 있다. 사용자가 만드는 tensor는 제외다. 사용자가 만든 tesnor는 grad_fn이 None이다. 

파생된 값들을 계산하기를 원한다면 tensor에서 **.backward()**를 호출할 수 있다. 

In [0]:
import torch

In [2]:
# Tensor를 만들고 requires_grad를 True 로 세팅하여 계산을 추적할 수 있도록 한다. 

x = torch.ones(2,2, requires_grad=True)

print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [3]:
# 간단한 tensor 작업을 해보자. 

y = x + 2

print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [4]:
# y tensor는 계산에 의해서 만들어졌기 때문에 grad_fn()을 가진다. 

print(y.grad_fn)

In [6]:
# y 에 다른 작업을 더 해보자.

z = y*y*3

out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [7]:
a = torch.rand(2,2)

a = ((a*3)/(a-1))
print(a.requires_grad)

a.requires_grad_(True)

print(a.requires_grad)

b = (a*a).sum()

print(b.grad_fn)

False
True


## Gradient

backprop을 해보자. **out**은 하나의 scalar를 포함하고 있기 때문에 **out.backward()**는 **out.backward(torch.tensor(1.))**과 같다. 

In [0]:
out.backward()

d(out)/dx 경사를 출력해보자.

In [9]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [10]:
x = torch.rand(3, requires_grad=True)
print(x)

tensor([0.8082, 0.5691, 0.8119], requires_grad=True)


In [11]:
y = x*2
while y.data.norm()<1000:
  y = y*2
  
print(y)

tensor([827.5931, 582.7769, 831.4139], grad_fn=<MulBackward0>)


In [13]:
print(x.requires_grad)

True


In [14]:
print((x**2).requires_grad)

True


In [15]:
with torch.no_grad():
  print((x**2).requires_grad)

False
